FINE TUNINING USING UNSLOTH

In [17]:
!apt-get install build-essential -y
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
 
# !mamba install --force-reinstall aiohttp -y
# !pip install -U "xformers<0.0.26" --index-url https://download.pytorch.org/whl/cu121
!pip install "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"

# # Temporary fix for https://github.com/huggingface/datasets/issues/6753
# !pip install datasets==2.16.0 fsspec==2023.10.0 gcsfs==2023.10.0

# import os
# os.environ["WANDB_DISABLED"] = "true"

# Installs Unsloth, Xformers (Flash Attention) and all other packages!
#!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
#!pip install --no-deps "xformers<0.0.26" "trl<0.9.0" peft accelerate bitsandbytes

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
0 upgraded, 0 newly installed, 0 to remove and 48 not upgraded.
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-9npzvjmp/unsloth_3a33e77ec45943e8bec4565a895b1c7e
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-9npzvjmp/unsloth_3a33e77ec45943e8bec4565a895b1c7e
  Resolved https://github.com/unslothai/unsloth.git to commit 12b437e12204532f82542c12ac1ab00d19e3ebbf
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [9]:
!pip install --no-deps "xformers<0.0.26" 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 MB 15.5 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: xformers
    Found existing installation: xformers 0.0.26.post1
    Uninstalling xformers-0.0.26.post1:
      Successfully uninstalled xformers-0.0.26.post1


In [1]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


In [3]:

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2-2b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = "", 
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
 
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  
    loftq_config = None,  
)


Unsloth: If you want to finetune Gemma 2, install flash-attn to make it faster!
To install flash-attn, do the below:

pip install --no-deps --upgrade "flash-attn>=2.6.3"
==((====))==  Unsloth 2024.8: Fast Gemma2 patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 3090. Max memory: 23.684 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.25.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth 2024.8 patched 26 layers with 26 QKV layers, 26 O layers and 26 MLP layers.


In [4]:
alpaca_prompt = """
### Instruction:
Your are an AI translator to translate the following English input sentence to Tamil sentence.

### Input:
{}

### Response:
{}"""

In [5]:
from datasets import load_dataset
dataset_name = "Hemanth-thunder/en_ta"
dataset = load_dataset(dataset_name, split="all")

In [6]:
dataset

Dataset({
    features: ['Unnamed: 0', 'en', 'ta'],
    num_rows: 288630
})

In [7]:


EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    inputs       = examples["en"]
    outputs      = examples["ta"]
    texts = []
    for input, output in zip( inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }



# Print the dataset
print(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

Dataset({
    features: ['Unnamed: 0', 'en', 'ta'],
    num_rows: 288630
})


In [8]:
print(dataset['text'][0])


### Instruction:
Your are an AI translator to translate the following English input sentence to Tamil sentence.

### Input:
MMA vice president Qazi Hussain Ahmad declared last month: 'We are not extremists.


### Response:
MMA கட்சியின் துணைத்தலைவர் க்வாஸி ஹுசேன் அகமத் சென்ற மாதம் பின்வருமாறு அறிவித்தார்: ``நாங்கள் தீவிரவாதிகள் அல்ல.
<eos>


In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 10000,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 100,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
max_steps is given, it will override any value given in num_train_epochs


In [10]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 3090. Max memory = 23.684 GB.
2.57 GB of memory reserved.


In [11]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 288,630 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 8 | Gradient Accumulation steps = 4
\        /    Total batch size = 32 | Total steps = 10,000
 "-____-"     Number of trainable parameters = 20,766,720


Step,Training Loss
100,1.497200
200,1.340100
300,1.308600
400,1.287500
500,1.269000
600,1.251000
700,1.230400
800,1.218500
900,1.205600
1000,1.198200


KeyboardInterrupt: 

In [13]:
FastLanguageModel.for_inference(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Gemma2ForCausalLM(
      (model): Gemma2Model(
        (embed_tokens): Embedding(256000, 2304)
        (layers): ModuleList(
          (0-25): 26 x Gemma2DecoderLayer(
            (self_attn): Gemma2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2304, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2304, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4bit(
  

In [14]:

mna_news_instruction1 = "Telangana CM Revanth Reddy Backs HYDRAA, Says Razing Actor Nagarjuna's Building As Per Law"
 

In [ ]:
inputs = tokenizer(
[
    alpaca_prompt.format(
       mna_news_instruction1,
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 200, use_cache = True)
tokenizer.batch_decode(outputs)[0].split("### Response:")[-1].strip()



In [ ]:
tokenizer.push_to_hub("Hemanth-thunder/model_gemma2b_mt-hi") 
model.save_pretrained("lora_model") # Local saving
model.push_to_hub("Hemanth-thunder/model_gemma2b_mt-hi") 

### For mergeing and Inference

In [ ]:
from unsloth import FastLanguageModel
import torch
from transformers import TextStreamer
max_seq_length = 1024 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.
 
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Hemanth-thunder/model_gemma2b_mt-hi_1k_steps", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,token = "",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference


In [15]:
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 512)

<bos>
### Instruction:
Your are an AI translator to translate the following English input sentence to Hindi sentence.

### Input:
Telangana CM Revanth Reddy Backs HYDRAA, Says Razing Actor Nagarjuna's Building As Per Law

### Response:
तेलंगाना के मुयमंत्री रेवंत reddy ने ह्यय्ड़्रेएहृ के साथ मिलकर नागार्जुन की इमारत को हटाने का समर्थन किया है।<eos>


In [3]:
model

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear4bit(in_features=2304, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2304, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=2304, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2304, bias=False)
          (rotary_emb): GemmaFixedRotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear4bit(in_features=2304, out_features=9216, bias=False)
          (up_proj): Linear4bit(in_features=2304, out_features=9216, bias=False)
          (down_proj): Linear4bit(in_features=9216, out_features=2304, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (post_attention_layernorm): Gemma2RMSN

In [7]:
inputs = tokenizer(
[
    alpaca_prompt.format(
       mna_news_instruction1,
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

In [8]:
inputs

{'input_ids': tensor([[     2,    108,   6176,  36142, 235292,    108,   6922,    708,    671,
          16481,  71783,    577,  19631,    573,   2412,   4645,   3772,  13060,
            577,  40686,  13060, 235265,    109,   6176,  11438, 235292,    108,
            651,   1703,    693,   3516,    682,    575,    573,  37083,   1554,
           4216,    109,   6176,  10567, 235292,    108]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}